In [1]:
!pip install transformers
import json
import copy
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import math
import torch
import torch.nn as nn
from torchtext.legacy import data
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from torch import Tensor 
import pandas as pd
import torch.nn.functional as F
from transformers import (WEIGHTS_NAME, BertConfig, BertModel, BertPreTrainedModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup)
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, RandomSampler, DataLoader, SequentialSampler
from sklearn.metrics import matthews_corrcoef, f1_score

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 38.4 MB/s 
     |████████████████████████████████| 61 kB 416 kB/s 
     |████████████████████████████████| 895 kB 63.5 MB/s 
     |████████████████████████████████| 3.3 MB 17.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
#from torchtext.data.metrics import bleu_score
from sklearn.model_selection import train_test_split
#from torch.utils.tensorboard import SummaryWriter
from google.colab import drive
drive.mount('/content/gdrive/')

%cd /content/gdrive/My Drive/Kaggle


Mounted at /content/gdrive/
/content/gdrive/My Drive/Kaggle


In [3]:
ADDITIONAL_SPECIAL_TOKENS = ['<e1>', "</e1>", "<e2>", "</e2>"]
def load_tokenizer():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenizer.add_special_tokens({"additional_special_tokens": ADDITIONAL_SPECIAL_TOKENS})
    return tokenizer

In [4]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, label_id, e1_mask, e2_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label_id = label_id
        self.e1_mask = e1_mask
        self.e2_mask = e2_mask

In [5]:
def convert_inputs_to_features(
    texts,
    label_id,
    max_seq_len,
    tokenizer,
    cls_token="[CLS]",
    sep_token="[SEP]",
    pad_token=0,
    mask_padding_with_zero=True,
):
    features = []
    #print(texts)
    for (ex_index, example) in enumerate(texts):

        #print(ex_index, example)
        tokens_a = tokenizer.tokenize(example)
        #print(tokens_a)

        e11_p = tokens_a.index("<e1>")  # the start position of entity1
        e12_p = tokens_a.index("</e1>")  # the end position of entity1
        e21_p = tokens_a.index("<e2>")  # the start position of entity2
        e22_p = tokens_a.index("</e2>")  # the end position of entity2

        # Replace the token
        tokens_a[e11_p] = "$"
        tokens_a[e12_p] = "$"
        tokens_a[e21_p] = "#"
        tokens_a[e22_p] = "#"

        # Add 1 because of the [CLS] token
        e11_p += 1
        e12_p += 1
        e21_p += 1
        e22_p += 1

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 2
        if len(tokens_a) > max_seq_len - special_tokens_count:
            tokens_a = tokens_a[: (max_seq_len - special_tokens_count)]

        tokens = [cls_token] + tokens_a + [sep_token]

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_len - len(input_ids)
        input_ids = input_ids + ([pad_token] * padding_length)
        attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)

        # e1 mask, e2 mask
        e1_mask = [0] * len(attention_mask)
        e2_mask = [0] * len(attention_mask)

        for i in range(e11_p, e12_p + 1):
            e1_mask[i] = 1
        for i in range(e21_p, e22_p + 1):
            e2_mask[i] = 1

        assert len(input_ids) == max_seq_len, "Error with input length {} vs {}".format(len(input_ids), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(
            len(attention_mask), max_seq_len
        )

        #print(len(input_ids), len(label_id))

        features.append(
            InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                label_id=label_id[ex_index],
                e1_mask=e1_mask,
                e2_mask=e2_mask,
            )
        )

    return features


In [6]:
'''
#!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
#!pip install inltk

from inltk.inltk import identify_language, tokenize, setup_language, reset_language_identifying_models
reset_language_identifying_models()
#setup_language('hi')
#setup_language('bn')
#setup_language('te')
#identify_language('hello, its english')
'''

#df = pd.read_csv('train.tsv', sep='\t')
df_en = pd.read_csv('en.tsv', sep='\t')
#df = pd.concat([df, df_en])

#inputs = []
'''
lang = identify_language(df['Sentence'].iloc[1]).run_until_complete()

for i in range(len(df)):
  lang = identify_language(df['Sentence'].iloc[i])
  sentence = ' '.join(tokenize(df['Sentence'].iloc[i], lang))
  inputs.append(df['NER1'].iloc[i]+'-'+df['NER2'].iloc[i]+' '+sentence)
'''
'''
for i in range(len(df)):
  inputs.append(df['NER1'].iloc[i]+'-'+df['NER2'].iloc[i]+' '+df['Sentence'].iloc[i])
df['CLS_sentence'] = inputs

df = df[['CLS_sentence', 'Relation']]
'''
labels = df_en['Relation'].unique().tolist()
labels_map = {}
for i in range(len(labels)):
  labels_map[labels[i]] = i

ids = []

for i in range(len(df_en)):
  ids.append(labels_map[df_en['Relation'].iloc[i]])
df_en['Label_id'] = ids

df = df_en[['Sentence', 'Label_id']]

df = df.sample(frac=1).reset_index(drop=True)

#print(df['CLS_sentence'].iloc[0])
              
#train, test = train_test_split(df, test_size = 0.2, shuffle = True, random_state = 42)

print()

In [7]:
def Load_Dataset():
    tokenizer = load_tokenizer()
    features = convert_inputs_to_features(df['Sentence'].iloc[:50].tolist(), df['Label_id'].iloc[:50].tolist(), 120, tokenizer)
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_e1_mask = torch.tensor([f.e1_mask for f in features], dtype=torch.long)  # add e1 mask
    all_e2_mask = torch.tensor([f.e2_mask for f in features], dtype=torch.long)  # add e2 mask
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

    #print(df['Label_id'].tolist()[0])
    #print(len([f.label_id for f in features][0]), len([f.input_ids for f in features][0]))
    print(all_input_ids.shape, all_label_ids.shape)

    dataset = TensorDataset(
        all_input_ids,
        all_attention_mask,
        all_label_ids,
        all_e1_mask,
        all_e2_mask
    )
    return dataset

In [8]:
train_dataset = Load_Dataset()

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

torch.Size([50, 120]) torch.Size([50])


In [9]:
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(BertForSequenceClassification, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.cls_dropout = nn.Dropout(0.1)  # dropout on CLS transformed token embedding
        self.ent_dropout = nn.Dropout(0.1)  # dropout on average entity embedding
        self.classifier = nn.Linear(config.hidden_size*3, self.config.num_labels)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None, e1_mask=None, e2_mask=None,
                position_ids=None, head_mask=None):
        outputs = self.bert(input_ids, position_ids=position_ids,
                            attention_mask=attention_mask, head_mask=head_mask)
        # for details, see https://huggingface.co/transformers/model_doc/bert.html#bertmodel

        pooled_output = outputs[1]  # sequence of hidden-states at the output of the last layer of the model
        sequence_output = outputs[0]  # last layer hidden-state of the first token of the sequence (classification token) further processed by a Linear layer and a Tanh activation function.

        def extract_entity(sequence_output, e_mask):
            extended_e_mask = e_mask.unsqueeze(1)
            extended_e_mask = torch.bmm(
                extended_e_mask.float(), sequence_output).squeeze(1)
            return extended_e_mask.float()

        e1_h = self.ent_dropout(extract_entity(sequence_output, e1_mask))
        e2_h = self.ent_dropout(extract_entity(sequence_output, e2_mask))
        context = self.cls_dropout(pooled_output)
        pooled_output = torch.cat([context, e1_h, e2_h], dim=-1)

        logits = self.classifier(pooled_output)

        # add hidden states and attention if they are here
        outputs = (logits,) + outputs[2:]
        #print(labels.shape)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            #print(logits.view(-1, self.num_labels).shape)
            #print(labels.view(-1).shape)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            #loss = loss_fct(logits.view(-1, self.num_labels), labels)
                
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device="cpu"

In [11]:
def simple_accuracy(preds, labels):
    return (preds == labels).mean()


def acc_and_f1(preds, labels, average='micro'):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds, average='micro')
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }

def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    return acc_and_f1(preds, labels)

In [12]:
def get_label(val):
    for key, value in labels_map.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [13]:
#%%writefile app.py
def save_checkpoint(state, filename="my_check.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(PATH, model, optimizer):
    print("=> Loading checkpoint")
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    epoch = checkpoint['epoch']
    return epoch, model, optimizer

In [14]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [15]:
adam_epsilon=1e-8
warmup_steps=0
max_steps = -1
learning_rate = 3e-5
batch_size = 16
num_train_epochs = 5


bertconfig = BertConfig.from_pretrained("bert-base-uncased", num_labels=len(labels))
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=bertconfig).to(device)


train_sampler = RandomSampler(train_dataset)

train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size= batch_size)

if max_steps > 0:
    t_total = max_steps
    num_train_epochs = max_steps // len(train_dataloader) + 1
else:
    t_total = len(train_dataloader) // num_train_epochs
        
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-3},
        {'params': [p for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters,
                      lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
def train(train_dataloader, optimizer, scheduler, train_dataset, model, tokenizer, num_train_epochs=5, max_grad_norm = 1.0):
    """ Train the model """

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    #train_iterator = trange(int(num_train_epochs), desc="Epoch")

    # Added here for reproductibility (even between python 2 and 3)
    set_seed(42)
    for _ in trange(int(num_train_epochs), desc="Epoch"): #train_iterator:
        #epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")): 
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':      batch[2],
                      'e1_mask': batch[3],
                      'e2_mask': batch[4],
                      }

            outputs = model(**inputs)
            # model outputs are always tuple in pytorch-transformers (see doc)
            loss = outputs[0]

            loss.backward()
            torch.nn.utils.clip_grad_norm_(
                model.parameters(), max_grad_norm)

            tr_loss += loss.item()

            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            model.zero_grad()
            global_step += 1

            if max_steps > 0 and global_step > max_steps:
                epoch_iterator.close()
                break
        if max_steps > 0 and global_step > max_steps:
            train_iterator.close()
            break
    return global_step, tr_loss / global_step

In [23]:
tokenizer = load_tokenizer()
global_step, loss = train(train_dataloader, optimizer, scheduler, train_dataset, model, tokenizer, num_train_epochs = 20, max_grad_norm=1.0)

Epoch: 100%|██████████| 20/20 [20:17<00:00, 60.86s/it]


In [18]:
loss

4.6253627955913545

In [19]:
def evaluate(model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    results = {}
    eval_dataset = Load_Dataset()

    eval_batch_size = 16
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(
        eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size, shuffle=False)

    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':      batch[2],
                      'e1_mask': batch[3],
                      'e2_mask': batch[4],
                      }
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(
                out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=1)
    for key in range(len(preds)):
      print(get_label(preds[key]))
    result = compute_metrics(preds, out_label_ids)
    results.update(result)
    print()
    print("***** Eval results {} *****".format(prefix))
    for key in sorted(result.keys()):
        print(f"{key} = {result[key]}")
    
    return result


In [20]:
results = {}
result = evaluate(model, tokenizer, prefix=global_step)
result = dict((k + '_{}'.format(global_step), v)
                          for k, v in result.items())
results.update(result)
results

torch.Size([50, 120]) torch.Size([50])


Evaluating: 100%|██████████| 4/4 [00:18<00:00,  4.53s/it]

original_language_of_film_or_TV_show
original_language_of_film_or_TV_show
original_language_of_film_or_TV_show
original_language_of_film_or_TV_show
position_held
position_held
occupation
original_language_of_film_or_TV_show
original_language_of_film_or_TV_show
child
original_language_of_film_or_TV_show
student_of
original_language_of_film_or_TV_show
position_held
position_held
original_language_of_film_or_TV_show
position_held
student_of
original_language_of_film_or_TV_show
original_language_of_film_or_TV_show
director
position_held
original_language_of_film_or_TV_show
subsidiary
original_language_of_film_or_TV_show
student_of
discoverer_or_inventor
original_language_of_film_or_TV_show
original_language_of_film_or_TV_show
discoverer_or_inventor
capital
discoverer_or_inventor
original_language_of_film_or_TV_show
director
occupation
original_language_of_film_or_TV_show
occupation
original_language_of_film_or_TV_show
subsidiary
position_held
original_language_of_film_or_TV_show
original_l

{'acc_20': 0.02, 'acc_and_f1_20': 0.02, 'f1_20': 0.02}

In [21]:
'''
for i in df['Label_id'].iloc[:50].tolist():
  print(get_label(i))
'''

"\nfor i in df['Label_id'].iloc[:50].tolist():\n  print(get_label(i))\n"

In [22]:
# %%writefile app.py
path = %pwd
PATH = os.path.join(path, 'my_check.pth.tar')
epoch, model, opt = load_checkpoint(PATH, model, opt)
df = pd.read_csv('valid.tsv', sep='\t')

outputs = []
for i in range(len(df)):
  outputs.append(predict(model, get_inp_sen_tensor(df['Sentence'].iloc[i]))[:-5])


NameError: ignored